In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/deepseek-r1-qwen-1.5b-mla.zip -d /content/

Archive:  /content/drive/MyDrive/deepseek-r1-qwen-1.5b-mla.zip
   creating: /content/deepseek-r1-qwen-1.5b-mla/
  inflating: /content/deepseek-r1-qwen-1.5b-mla/config.json  
  inflating: /content/deepseek-r1-qwen-1.5b-mla/generation_config.json  
  inflating: /content/deepseek-r1-qwen-1.5b-mla/model.safetensors  
  inflating: /content/deepseek-r1-qwen-1.5b-mla/special_tokens_map.json  
  inflating: /content/deepseek-r1-qwen-1.5b-mla/tokenizer.json  
  inflating: /content/deepseek-r1-qwen-1.5b-mla/tokenizer_config.json  


In [ ]:
import os
# Adjust this path to where you uploaded it in your Drive
local_model_path = "/content/deepseek-r1-qwen-1.5b-mla"
if not os.path.isdir(local_model_path):
    print(f"Model directory not found at {local_model_path}. Please check the path.")
    local_model_path = None
else:
    print(f"Using model from Google Drive: {local_model_path}")
    assert os.path.exists(os.path.join(local_model_path, "config.json")), "config.json not found in Drive path!"

Using model from Google Drive: /content/deepseek-r1-qwen-1.5b-mla


In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch # Import torch to specify dtype if needed

# 1. Load the model and tokenizer
max_seq_length = 2048 # Or your desired maximum sequence length
# Automatically set dtype for a slight speedup depending on your GPU
dtype = None
load_in_4bit = False # Set to True to load in 4-bit for reduced memory usage
LOCAL_MODEL_PATH = "/content/deepseek-r1-qwen-1.5b-mla"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOCAL_MODEL_PATH,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Optional: Enable faster inference - Unsloth often handles this automatically
# FastLanguageModel.for_inference(model)

# 2. Prepare your input prompt
# Use the chat template appropriate for Qwen models.
messages = [
    {"role": "system", "content": "You are a helpful AI assistant. The question will be in Polish language."},
    {"role": "user", "content": "Cześć! Jak się masz?"},
]

# Apply the chat template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize the input prompt and move to GPU
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# 3. Perform inference
# Use TextStreamer for streaming the output in Colab
text_streamer = TextStreamer(tokenizer)

print("Generating response...")
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=150, use_cache=True) # Adjust max_new_tokens as needed

==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Some weights of the model checkpoint at /content/deepseek-r1-qwen-1.5b-mla were not used when initializing Qwen2ForCausalLM: ['model.layers.0.self_attn.k_compress.weight', 'model.layers.0.self_attn.k_decompress.weight', 'model.layers.0.self_attn.v_compress.weight', 'model.layers.0.self_attn.v_decompress.weight', 'model.layers.1.self_attn.k_compress.weight', 'model.layers.1.self_attn.k_decompress.weight', 'model.layers.1.self_attn.v_compress.weight', 'model.layers.1.self_attn.v_decompress.weight', 'model.layers.10.self_attn.k_compress.weight', 'model.layers.10.self_attn.k_decompress.weight', 'model.layers.10.self_attn.v_compress.weight', 'model.layers.10.self_attn.v_decompress.weight', 'model.layers.11.self_attn.k_compress.weight', 'model.layers.11.self_attn.k_decompress.weight', 'model.layers.11.self_attn.v_compress.weight', 'model.layers.11.self_attn.v_decompress.weight', 'model.layers.12.self_attn.k_compress.weight', 'model.layers.12.self_attn.k_decompress.weight', 'model.layers.12.s

Generating response...
<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜>You are a helpful AI assistant. The question will be in Polish language.<｜User｜>Cześć! Jak się masz?<｜Assistant｜><think>
Okay, so I need to figure out how to respond to the user's question in Polish. They asked, "Cześć! How are you?" which translates to "Hello! How are you?" in English. My task is to respond in Polish. 

First, I should acknowledge their greeting. In Polish, "Cześć" is the appropriate greeting, so I'll use that. Then, I need to respond in a friendly manner. It's important to keep the tone positive and welcoming.

I should also provide information about their situation. Since they just said "how are you," I can offer to help them out. That way, I'm being helpful and inviting them to ask more questions if they need it.

Putting it all


In [ ]:
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

MODEL_NAME = "/content/deepseek-r1-qwen-1.5b-mla"
MAX_LEN    = 4096

raw_ds = load_dataset("emplocity/owca", split="train")

template_tokenizer = get_chat_template(tokenizer, chat_template="alpaca")
def format_as_chat(example):
    messages = []
    instruction = example.get("instruction", "").strip()
    input_text = example.get("input", "").strip()
    output = example.get("output", "").strip()

    if input_text:
        messages.append({"role": "user", "content": f"{instruction}\n\n{input_text}"})
    else:
        messages.append({"role": "user", "content": instruction})

    messages.append({"role": "assistant", "content": output})

    return {
        "text": tokenizer.apply_chat_template(messages, tokenize=False)
    }

train_ds = raw_ds.map(format_as_chat, remove_columns=raw_ds.column_names)

README.md:   0%|          | 0.00/2.30k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


alpaca_data_cleaned_pl_emplocity.json:   0%|          | 0.00/22.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51713 [00:00<?, ? examples/s]

Map:   0%|          | 0/51713 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
import torch, time, json
from unsloth import UnslothTrainingArguments

#FastLanguageModel.for_inference(model)   # Flash-Attn 2 path

peft_model = FastLanguageModel.get_peft_model(
        model, r=16, lora_alpha=16,
        target_modules=["q_proj","k_proj","v_proj","o_proj",
                        "gate_proj","up_proj","down_proj"])

t0 = time.time()
args = UnslothTrainingArguments(
    output_dir="out_mla",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=20,
    save_strategy="steps",
    save_steps=1000
)
trainer = SFTTrainer(
        model=peft_model, tokenizer=tokenizer, train_dataset=train_ds,
        max_seq_length=MAX_LEN,
        args=args)
trainer.train()
json.dump({"wall_time_s": time.time()-t0}, open("out_mha/time.json","w"))


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/51713 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,713 | Num Epochs = 1 | Total steps = 12,928
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 18,464,768/1,795,552,768 (1.03% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s184339 (s184339-politechnika-gda-ska) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,3.490400
40,2.714000
60,2.357800
80,2.309200
100,2.029500
120,2.182200
140,2.105900
160,2.121100
180,2.197300
200,2.276000


Unsloth: Will smartly offload gradients to save VRAM!


KeyboardInterrupt: 

In [ ]:
!rm -rf my_lora_adapters/

In [ ]:

from google.colab import files # For downloading later
# Define a path in your Colab environment to save the adapters
adapter_save_path = "/content/my_lora_adapters"
os.makedirs(adapter_save_path, exist_ok=True)

try:
    print(f"Saving PEFT adapters to {adapter_save_path}...")
    # This saves the LoRA adapters and the adapter_config.json
    # For Unsloth models, this is the standard way to save your LoRA fine-tune
    peft_model.save_pretrained(adapter_save_path)

    # It's also good practice to save the tokenizer alongside your adapters
    if 'tokenizer' in locals() and tokenizer is not None:
        tokenizer.save_pretrained(adapter_save_path)
        print(f"Tokenizer saved to {adapter_save_path}")
    else:
        # If you loaded the model with FastLanguageModel, the tokenizer might be attached to it
        # Or you might have it in a different variable.
        # For Unsloth, often the tokenizer is loaded with the model:
        # model, tokenizer = FastLanguageModel.from_pretrained(...)
        # So, ensure your 'tokenizer' variable is correctly referenced.
        print("Tokenizer variable not found or is None. Please save it manually if needed.")

    print("PEFT adapters saved successfully.")

    # To download the saved adapters from Colab:
    # Option A: Download individual files (if you know them, e.g., adapter_model.safetensors, adapter_config.json)
    # files.download(os.path.join(adapter_save_path, "adapter_model.safetensors"))
    # files.download(os.path.join(adapter_save_path, "adapter_config.json"))

    # Option B: Zip the entire directory and download the zip (more convenient)
    archive_path = f"{adapter_save_path}.zip"
    !zip -r {archive_path} {adapter_save_path}
    print(f"Adapters zipped to {archive_path}")
    files.download(archive_path)

except Exception as e:
    print(f"Error saving PEFT adapters: {e}")
    import traceback
    traceback.print_exc()

Saving PEFT adapters to /content/my_lora_adapters...
Tokenizer saved to /content/my_lora_adapters
PEFT adapters saved successfully.
  adding: content/my_lora_adapters/ (stored 0%)
  adding: content/my_lora_adapters/adapter_model.safetensors (deflated 7%)
  adding: content/my_lora_adapters/special_tokens_map.json (deflated 70%)
  adding: content/my_lora_adapters/tokenizer.json (deflated 81%)
  adding: content/my_lora_adapters/tokenizer_config.json (deflated 84%)
  adding: content/my_lora_adapters/adapter_config.json (deflated 57%)
  adding: content/my_lora_adapters/README.md (deflated 66%)
Adapters zipped to /content/my_lora_adapters.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pwd

/content


In [ ]:
!unzip /content/my_lora_adapters_mla.zip

Archive:  /content/my_lora_adapters_mla.zip
   creating: content/my_lora_adapters/
  inflating: content/my_lora_adapters/adapter_model.safetensors  
  inflating: content/my_lora_adapters/special_tokens_map.json  
  inflating: content/my_lora_adapters/tokenizer.json  
  inflating: content/my_lora_adapters/tokenizer_config.json  
  inflating: content/my_lora_adapters/adapter_config.json  
  inflating: content/my_lora_adapters/README.md  


In [ ]:
!unzip /content/my_lora_adapters_gqa.zip

Archive:  /content/my_lora_adapters_gqa.zip
   creating: content/my_lora_adapters/
  inflating: content/my_lora_adapters/special_tokens_map.json  
  inflating: content/my_lora_adapters/tokenizer.json  
  inflating: content/my_lora_adapters/adapter_model.safetensors  
  inflating: content/my_lora_adapters/adapter_config.json  
  inflating: content/my_lora_adapters/README.md  
  inflating: content/my_lora_adapters/tokenizer_config.json  


In [ ]:
from unsloth import FastLanguageModel # Ensure it's imported
import torch

# Assume 'model' and 'tokenizer' are already loaded and 'model' is your fine-tuned PeftModel
MAX_LEN = 2048 # As defined in your training
adapter_path = "/content/content/my_lora_adapters"
dtype = None
load_in_4bit = False # Set to True to load in 4-bit for reduced memory usage
LOCAL_MODEL_PATH = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOCAL_MODEL_PATH,
    max_seq_length = MAX_LEN,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"Loading adapter from: {adapter_path}")
model.load_adapter(adapter_path)



# Set model to evaluation mode
if hasattr(model, 'eval'):
    model.eval()
    print("Model set to evaluation mode.")

# Enable Unsloth's fast inference path
# This should be called on the PeftModel directly
FastLanguageModel.for_inference(model)
print("Unsloth's FastLanguageModel.for_inference(model) applied.")

==((====))==  Unsloth 2025.5.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

Loading adapter from: /content/content/my_lora_adapters
Model set to evaluation mode.
Unsloth's FastLanguageModel.for_inference(model) applied.


In [ ]:
from unsloth import FastLanguageModel # Ensure it's imported
import torch

# Assume 'model' and 'tokenizer' are already loaded and 'model' is your fine-tuned PeftModel
MAX_LEN = 2048 # As defined in your training
adapter_path = "/content/content/my_lora_adapters"
dtype = None
load_in_4bit = False # Set to True to load in 4-bit for reduced memory usage
LOCAL_MODEL_PATH = "/content/deepseek-r1-qwen-1.5b-mla"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOCAL_MODEL_PATH,
    max_seq_length = MAX_LEN,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"Loading adapter from: {adapter_path}")
model.load_adapter(adapter_path)



# Set model to evaluation mode
if hasattr(model, 'eval'):
    model.eval()
    print("Model set to evaluation mode.")

# Enable Unsloth's fast inference path
# This should be called on the PeftModel directly
FastLanguageModel.for_inference(model)
print("Unsloth's FastLanguageModel.for_inference(model) applied.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Some weights of the model checkpoint at /content/deepseek-r1-qwen-1.5b-mla were not used when initializing Qwen2ForCausalLM: ['model.layers.0.self_attn.k_compress.weight', 'model.layers.0.self_attn.k_decompress.weight', 'model.layers.0.self_attn.v_compress.weight', 'model.layers.0.self_attn.v_decompress.weight', 'model.layers.1.self_attn.k_compress.weight', 'model.layers.1.self_attn.k_decompress.weight', 'model.layers.1.self_attn.v_compress.weight', 'model.layers.1.self_attn.v_decompress.weight', 'model.layers.10.self_attn.k_compress.weight', 'model.layers.10.self_attn.k_decompress.weight', 'model.layers.10.self_attn.v_compress.weight', 'model.layers.10.self_attn.v_decompress.weight', 'model.layers.11.self_attn.k_compress.weight', 'model.layers.11.self_attn.k_decompress.weight', 'model.layers.11.self_attn.v_compress.weight', 'model.layers.11.self_attn.v_decompress.weight', 'model

Loading adapter from: /content/content/my_lora_adapters
Model set to evaluation mode.
Unsloth's FastLanguageModel.for_inference(model) applied.


In [ ]:
from unsloth import FastLanguageModel # Ensure it's imported
import torch

# Assume 'model' and 'tokenizer' are already loaded and 'model' is your fine-tuned PeftModel
MAX_LEN = 2048 # As defined in your training
adapter_path = "/content/content/my_lora_adapters_gqa"
dtype = None
load_in_4bit = False # Set to True to load in 4-bit for reduced memory usage
LOCAL_MODEL_PATH = "/content/deepseek-r1-qwen-1.5b-mla"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOCAL_MODEL_PATH,
    max_seq_length = MAX_LEN,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"Loading adapter from: {adapter_path}")
model.load_adapter(adapter_path)



# Set model to evaluation mode
if hasattr(model, 'eval'):
    model.eval()
    print("Model set to evaluation mode.")

# Enable Unsloth's fast inference path
# This should be called on the PeftModel directly
FastLanguageModel.for_inference(model)
print("Unsloth's FastLanguageModel.for_inference(model) applied.")

In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.10.0
    Uninstalling fsspec-2023.10.0:
      Successfully uninstalled fsspec-2023.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 whic

In [ ]:
from datasets import load_dataset, Dataset

print("Loading and preparing evaluation dataset slice (last 5000 samples from OWCA)...")
raw_ds_full = load_dataset("emplocity/owca", split="train")

def is_valid_entry(example):
  output_text = example.get("output")
  if output_text == '':
    return False
  else:
    return True

filtered_ds = raw_ds_full.filter(is_valid_entry)
total_raw_samples = len(filtered_ds)
eval_samples_count = 5000 # Number of samples to evaluate
# eval_samples_count = 100 # Use a smaller number for quicker testing first!

raw_eval_slice = None
if total_raw_samples >= eval_samples_count:
    eval_indices = range(total_raw_samples - eval_samples_count, total_raw_samples)
    raw_eval_slice = filtered_ds.select(eval_indices)
elif total_raw_samples > 0:
    raw_eval_slice = filtered_ds # Use all if less than desired count
    print(f"Warning: Dataset has only {total_raw_samples} samples. Evaluating on all available.")
else:
    print("Error: Raw dataset 'emplocity/owca' is empty or could not be loaded.")

if raw_eval_slice:
    print(f"Prepared evaluation slice with {len(raw_eval_slice)} samples.")
    # You can inspect a sample: print(raw_eval_slice[0])
else:
    print("Evaluation cannot proceed without data.")

Loading and preparing evaluation dataset slice (last 5000 samples from OWCA)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.30k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


alpaca_data_cleaned_pl_emplocity.json:   0%|          | 0.00/22.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51713 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51713 [00:00<?, ? examples/s]

Prepared evaluation slice with 5000 samples.


In [ ]:
# WITHOUT ADAPTERS - GQA VERSION
from unsloth import FastLanguageModel # Ensure it's imported
import torch

# Assume 'model' and 'tokenizer' are already loaded and 'model' is your fine-tuned PeftModel
MAX_LEN = 2048 # As defined in your training
#adapter_path = "/content/content/my_lora_adapters"
dtype = None
load_in_4bit = False # Set to True to load in 4-bit for reduced memory usage
LOCAL_MODEL_PATH = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOCAL_MODEL_PATH,
    max_seq_length = MAX_LEN,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

#print(f"Loading adapter from: {adapter_path}")
#model.load_adapter(adapter_path)



# Set model to evaluation mode
if hasattr(model, 'eval'):
    model.eval()
    print("Model set to evaluation mode.")

# Enable Unsloth's fast inference path
# This should be called on the PeftModel directly
FastLanguageModel.for_inference(model)
print("Unsloth's FastLanguageModel.for_inference(model) applied.")

==((====))==  Unsloth 2025.5.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model set to evaluation mode.
Unsloth's FastLanguageModel.for_inference(model) applied.


In [ ]:
# WITHOUT ADAPTERS - MLA VERSION
from unsloth import FastLanguageModel # Ensure it's imported
import torch

# Assume 'model' and 'tokenizer' are already loaded and 'model' is your fine-tuned PeftModel
MAX_LEN = 2048 # As defined in your training
#adapter_path = "/content/content/my_lora_adapters_gqa"
dtype = None
load_in_4bit = False # Set to True to load in 4-bit for reduced memory usage
LOCAL_MODEL_PATH = "/content/deepseek-r1-qwen-1.5b-mla"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LOCAL_MODEL_PATH,
    max_seq_length = MAX_LEN,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

#print(f"Loading adapter from: {adapter_path}")
#model.load_adapter(adapter_path)



# Set model to evaluation mode
if hasattr(model, 'eval'):
    model.eval()
    print("Model set to evaluation mode.")

# Enable Unsloth's fast inference path
# This should be called on the PeftModel directly
FastLanguageModel.for_inference(model)
print("Unsloth's FastLanguageModel.for_inference(model) applied.")

==((====))==  Unsloth 2025.5.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Some weights of the model checkpoint at /content/deepseek-r1-qwen-1.5b-mla were not used when initializing Qwen2ForCausalLM: ['model.layers.0.self_attn.k_compress.weight', 'model.layers.0.self_attn.k_decompress.weight', 'model.layers.0.self_attn.v_compress.weight', 'model.layers.0.self_attn.v_decompress.weight', 'model.layers.1.self_attn.k_compress.weight', 'model.layers.1.self_attn.k_decompress.weight', 'model.layers.1.self_attn.v_compress.weight', 'model.layers.1.self_attn.v_decompress.weight', 'model.layers.10.self_attn.k_compress.weight', 'model.layers.10.self_attn.k_decompress.weight', 'model.layers.10.self_attn.v_compress.weight', 'model.layers.10.self_attn.v_decompress.weight', 'model.layers.11.self_attn.k_compress.weight', 'model.layers.11.self_attn.k_decompress.weight', 'model.layers.11.self_attn.v_compress.weight', 'model.layers.11.self_attn.v_decompress.weight', 'model.layers.12.self_attn.k_compress.weight', 'model.layers.12.self_attn.k_decompress.weight', 'model.layers.12.s

Model set to evaluation mode.
Unsloth's FastLanguageModel.for_inference(model) applied.


In [ ]:
import pandas as pd
from tqdm.auto import tqdm

generated_responses = []
reference_responses = []

# Adjusted token length for better output completeness
MAX_NEW_TOKENS_EVAL = 512
quick_i = 0

if raw_eval_slice:
    print(f"\nGenerating responses for {len(raw_eval_slice)} evaluation samples (batch size = 1)...")

    for example in tqdm(raw_eval_slice):
        quick_i += 1
        if quick_i > 21:
            break

        instruction = example.get("instruction", "").strip()
        input_text = example.get("input", "").strip()
        reference_output = example.get("output", "").strip()
        reference_responses.append(reference_output)

        # Create prompt
        if input_text:
            content = f"{instruction}\n\n{input_text}"
        else:
            content = instruction

        messages = [{"role": "user", "content": content}]
        prompt_string = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Tokenize
        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LEN
        ).to("cuda")

        # Generate
        with torch.no_grad():
            outputs_tokens = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS_EVAL,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.8,
                top_p=0.9
            )

        decoded_output = tokenizer.decode(outputs_tokens[0], skip_special_tokens=True).strip()
        generated_responses.append(decoded_output)



Generating responses for 5000 evaluation samples (batch size = 1)...


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
# For GQA version
import pandas as pd
from tqdm.auto import tqdm

generated_responses = []
reference_responses = []

# Adjusted token length for better output completeness
MAX_NEW_TOKENS_EVAL = 512
quick_i = 0

if raw_eval_slice:
    print(f"\nGenerating responses for {len(raw_eval_slice)} evaluation samples (batch size = 1)...")

    for example in tqdm(raw_eval_slice):
        quick_i += 1
        if quick_i > 21:
            break

        instruction = example.get("instruction", "").strip()
        input_text = example.get("input", "").strip()
        reference_output = example.get("output", "").strip()
        reference_responses.append(reference_output)

        # Create prompt
        if input_text:
            content = f"{instruction}\n\n{input_text}"
        else:
            content = instruction

        messages = [{"role": "user", "content": content}]
        prompt_string = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Tokenize
        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LEN
        ).to("cuda")

        # Generate
        with torch.no_grad():
            outputs_tokens = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS_EVAL,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.8,
                top_p=0.9
            )

        decoded_output = tokenizer.decode(outputs_tokens[0], skip_special_tokens=True).strip()
        generated_responses.append(decoded_output)



Generating responses for 5000 evaluation samples (batch size = 1)...


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
import json
with open("generated_mla.json", 'w') as f:
    json.dump(generated_responses, f, indent=2)

In [ ]:
import json
with open("generated_gqa.json", 'w') as f:
    json.dump(generated_responses, f, indent=2)

In [ ]:
with open("reference.json", 'w') as f:
    json.dump(reference_responses, f, indent=2)

In [ ]:
with open("reference_gqa.json", 'w') as f:
    json.dump(reference_responses, f, indent=2)

In [ ]:
import json
with open("generated_gqa_NO_FINETUNE.json", 'w') as f:
    json.dump(generated_responses, f, indent=2)

In [ ]:
import json
with open("generated_mla_NO_FINETUNE.json", 'w') as f:
    json.dump(generated_responses, f, indent=2)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install evaluate rouge_score sacrebleu nltk bert_score --quiet
import nltk
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True) # Often needed for METEOR if you use it
nltk.download('omw-1.4', quiet=True) # Also for wordnet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   

True

In [ ]:
import evaluate as hf_evaluate # Renamed to avoid conflict if you have 'evaluate' variable

if generated_responses and reference_responses:
    print("\nCalculating ROUGE scores...")
    rouge_metric = hf_evaluate.load('rouge')
    rouge_scores = rouge_metric.compute(predictions=generated_responses, references=reference_responses)
    print(rouge_scores)

    print("\nCalculating BLEU score...")
    bleu_metric = hf_evaluate.load('sacrebleu') # SacreBLEU is often preferred
    # BLEU expects references to be a list of lists (for multiple references per prediction)
    # For OWCA, we typically have one reference.
    bleu_scores = bleu_metric.compute(predictions=generated_responses, references=[[r] for r in reference_responses])
    print(f"BLEU Score: {bleu_scores['score']:.4f}")
    # print(f"BLEU Details: {bleu_scores}")


    # Optional: METEOR (requires wordnet and punkt to be downloaded by nltk)
    print("\nCalculating METEOR score...")
    try:
        meteor_metric = hf_evaluate.load('meteor')
        meteor_scores = meteor_metric.compute(predictions=generated_responses, references=reference_responses)
        print(meteor_scores)
    except Exception as e:
        print(f"Could not compute METEOR score: {e}. Ensure NLTK's wordnet is downloaded.")

else:
    print("Skipping metric calculation as no responses were generated.")


Calculating ROUGE scores...
{'rouge1': np.float64(0.12351887500357472), 'rouge2': np.float64(0.040468939195317355), 'rougeL': np.float64(0.0844923439781498), 'rougeLsum': np.float64(0.10740291519088313)}

Calculating BLEU score...


BLEU Score: 2.0569

Calculating METEOR score...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': np.float64(0.16184025882597736)}

Calculating BERTScore (this might take a while)...
Could not compute BERTScore: name 'bertscore_metric' is not defined


In [ ]:
!pip install evaluate bert-score

In [ ]:
import evaluate as hf_evaluate

# Load BERTScore metric
bertscore_metric = hf_evaluate.load("bertscore")

# Compute BERTScore
bertscore_results = bertscore_metric.compute(
    predictions=generated_responses,
    references=reference_responses,
    lang="pl"  # Use "pl" for Polish
)

# Print averaged scores
print({
    "bertscore_precision": sum(bertscore_results["precision"]) / len(bertscore_results["precision"]),
    "bertscore_recall": sum(bertscore_results["recall"]) / len(bertscore_results["recall"]),
    "bertscore_f1": sum(bertscore_results["f1"]) / len(bertscore_results["f1"]),
})


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

{'bertscore_precision': 0.558929918777375, 'bertscore_recall': 0.6784964374133519, 'bertscore_f1': 0.6113560625485012}


In [ ]:
# GQA
import evaluate as hf_evaluate # Renamed to avoid conflict if you have 'evaluate' variable

if generated_responses and reference_responses:
    print("\nCalculating ROUGE scores...")
    rouge_metric = hf_evaluate.load('rouge')
    rouge_scores = rouge_metric.compute(predictions=generated_responses, references=reference_responses)
    print(rouge_scores)

    print("\nCalculating BLEU score...")
    bleu_metric = hf_evaluate.load('sacrebleu') # SacreBLEU is often preferred
    # BLEU expects references to be a list of lists (for multiple references per prediction)
    # For OWCA, we typically have one reference.
    bleu_scores = bleu_metric.compute(predictions=generated_responses, references=[[r] for r in reference_responses])
    print(f"BLEU Score: {bleu_scores['score']:.4f}")
    # print(f"BLEU Details: {bleu_scores}")


    # Optional: METEOR (requires wordnet and punkt to be downloaded by nltk)
    print("\nCalculating METEOR score...")
    try:
        meteor_metric = hf_evaluate.load('meteor')
        meteor_scores = meteor_metric.compute(predictions=generated_responses, references=reference_responses)
        print(meteor_scores)
    except Exception as e:
        print(f"Could not compute METEOR score: {e}. Ensure NLTK's wordnet is downloaded.")
        import evaluate as hf_evaluate

    # Load BERTScore metric
    bertscore_metric = hf_evaluate.load("bertscore")

    # Compute BERTScore
    bertscore_results = bertscore_metric.compute(
        predictions=generated_responses,
        references=reference_responses,
        lang="pl"  # Use "pl" for Polish
    )

    # Print averaged scores
    print({
        "bertscore_precision": sum(bertscore_results["precision"]) / len(bertscore_results["precision"]),
        "bertscore_recall": sum(bertscore_results["recall"]) / len(bertscore_results["recall"]),
        "bertscore_f1": sum(bertscore_results["f1"]) / len(bertscore_results["f1"]),
    })


else:
    print("Skipping metric calculation as no responses were generated.")


Calculating ROUGE scores...
{'rouge1': np.float64(0.11603783993709156), 'rouge2': np.float64(0.04094068920891193), 'rougeL': np.float64(0.08550943507141265), 'rougeLsum': np.float64(0.09549420087136304)}

Calculating BLEU score...
BLEU Score: 2.4060

Calculating METEOR score...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': np.float64(0.17148179476465242)}
{'bertscore_precision': 0.5602291425069174, 'bertscore_recall': 0.6764205268451146, 'bertscore_f1': 0.6101714046228499}


In [ ]:
with open("generated_mla.json", "r", encoding="utf-8") as f:
    raw_json = f.read()

# Decode Unicode escape sequences
decoded = raw_json.encode().decode('unicode_escape')
print(decoded)


[
  "<｜User｜>Wyjaśnij dlaczego Sztuczna Inteligencja jest korzystna w naszym codziennym życiu.<｜Assistant｜><think>
Dlaczego w naszym codziennym życiu ważne jest Sztuczna Inteligencja?

Sztuczna Inteligencja (AI) to dziedzina sztucznej inteligencji, która wykorzystuje algorytmy i technologie do analizowania i rozumienia danych, a także rozwiązywania problemów bez konieczności mechanycznej widzenia. W naszym codziennym życiu AI ma kilka ważnych konsekwencji.

Z jednej strony, AI pomaga nam rozumieć, kiedy się się gubić, co wyróżnia, jakie informacje oczekujemy i kiedy mamy do czynienia z niewygodami. Dzięki AI, możemy zrozumieć, jakie dane mamy, jakie informacje mamy i jakie czynniki wpływają na nasze decyzje. Dzięki AI, możemy zrozumieć, jak doświadczenia i informacje są niesamowicie związkane z naszymi mentallyymi doświadczeniami, co pozwala nam lepiej rozumieć, jak się wyróżnia i kiedy mamy do czynienia z niewygodami.

Z drugiej strony, AI pomaga nam podejmować lepsze decyzje i podejm

In [ ]:
# GQA NO FINETUNE
import evaluate as hf_evaluate

if generated_responses and reference_responses:
    print("\nCalculating ROUGE scores...")
    rouge_metric = hf_evaluate.load('rouge')
    rouge_scores = rouge_metric.compute(predictions=generated_responses, references=reference_responses)
    print(rouge_scores)

    print("\nCalculating BLEU score...")
    bleu_metric = hf_evaluate.load('sacrebleu')
    bleu_scores = bleu_metric.compute(predictions=generated_responses, references=[[r] for r in reference_responses])
    print(f"BLEU Score: {bleu_scores['score']:.4f}")



    print("\nCalculating METEOR score...")
    try:
        meteor_metric = hf_evaluate.load('meteor')
        meteor_scores = meteor_metric.compute(predictions=generated_responses, references=reference_responses)
        print(meteor_scores)
    except Exception as e:
        print(f"Could not compute METEOR score: {e}. Ensure NLTK's wordnet is downloaded.")
        import evaluate as hf_evaluate

    # Load BERTScore metric
    bertscore_metric = hf_evaluate.load("bertscore")

    bertscore_results = bertscore_metric.compute(
        predictions=generated_responses,
        references=reference_responses,
        lang="pl"
    )

    print({
        "bertscore_precision": sum(bertscore_results["precision"]) / len(bertscore_results["precision"]),
        "bertscore_recall": sum(bertscore_results["recall"]) / len(bertscore_results["recall"]),
        "bertscore_f1": sum(bertscore_results["f1"]) / len(bertscore_results["f1"]),
    })


else:
    print("Skipping metric calculation as no responses were generated.")


Calculating ROUGE scores...
{'rouge1': np.float64(0.06581692259467123), 'rouge2': np.float64(0.019581263460238526), 'rougeL': np.float64(0.05000393409207865), 'rougeLsum': np.float64(0.05680677013949805)}

Calculating BLEU score...
BLEU Score: 1.1566

Calculating METEOR score...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': np.float64(0.12962420648246498)}
{'bertscore_precision': 0.5338826108546484, 'bertscore_recall': 0.6512548157146999, 'bertscore_f1': 0.5850780166330791}


In [ ]:
# MLA NO FINETUNE
import evaluate as hf_evaluate # Renamed to avoid conflict if you have 'evaluate' variable

if generated_responses and reference_responses:
    print("\nCalculating ROUGE scores...")
    rouge_metric = hf_evaluate.load('rouge')
    rouge_scores = rouge_metric.compute(predictions=generated_responses, references=reference_responses)
    print(rouge_scores)

    print("\nCalculating BLEU score...")
    bleu_metric = hf_evaluate.load('sacrebleu') # SacreBLEU is often preferred
    # BLEU expects references to be a list of lists (for multiple references per prediction)
    # For OWCA, we typically have one reference.
    bleu_scores = bleu_metric.compute(predictions=generated_responses, references=[[r] for r in reference_responses])
    print(f"BLEU Score: {bleu_scores['score']:.4f}")
    # print(f"BLEU Details: {bleu_scores}")


    # Optional: METEOR (requires wordnet and punkt to be downloaded by nltk)
    print("\nCalculating METEOR score...")
    try:
        meteor_metric = hf_evaluate.load('meteor')
        meteor_scores = meteor_metric.compute(predictions=generated_responses, references=reference_responses)
        print(meteor_scores)
    except Exception as e:
        print(f"Could not compute METEOR score: {e}. Ensure NLTK's wordnet is downloaded.")
        import evaluate as hf_evaluate

    # Load BERTScore metric
    bertscore_metric = hf_evaluate.load("bertscore")

    # Compute BERTScore
    bertscore_results = bertscore_metric.compute(
        predictions=generated_responses,
        references=reference_responses,
        lang="pl"  # Use "pl" for Polish
    )

    # Print averaged scores
    print({
        "bertscore_precision": sum(bertscore_results["precision"]) / len(bertscore_results["precision"]),
        "bertscore_recall": sum(bertscore_results["recall"]) / len(bertscore_results["recall"]),
        "bertscore_f1": sum(bertscore_results["f1"]) / len(bertscore_results["f1"]),
    })


else:
    print("Skipping metric calculation as no responses were generated.")


Calculating ROUGE scores...
{'rouge1': np.float64(0.06253807438936947), 'rouge2': np.float64(0.02241829236139612), 'rougeL': np.float64(0.05116521214436198), 'rougeLsum': np.float64(0.056380567571886364)}

Calculating BLEU score...
BLEU Score: 1.0413

Calculating METEOR score...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': np.float64(0.10927622974706123)}
{'bertscore_precision': 0.5248591346400124, 'bertscore_recall': 0.645571387949444, 'bertscore_f1': 0.5771635018643879}
